In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [70]:
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv('../data/processed/korean_sentiment_dataset.csv')
print(f"데이터 크기: {df.shape}")
print(f"라벨 분포:\n{df['label'].value_counts().sort_index()}")

emotion_map = {0: '우울', 1: '불안', 2: '정상'}
print("감정 매핑:", emotion_map)
print("="*50)

데이터 크기: (3000, 2)
라벨 분포:
0    1000
1    1000
2    1000
Name: label, dtype: int64
감정 매핑: {0: '우울', 1: '불안', 2: '정상'}


In [71]:
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"훈련: {len(X_train)}, 테스트: {len(X_test)}")

훈련: 2400, 테스트: 600


In [72]:
tfidf = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.8,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(f"TF-IDF 특성 수: {X_train_tfidf.shape[1]}")
print(f"훈련 벡터 크기: {X_train_tfidf.shape}")
print(f"테스트 벡터 크기: {X_test_tfidf.shape}")


TF-IDF 특성 수: 1217
훈련 벡터 크기: (2400, 1217)
테스트 벡터 크기: (600, 1217)


In [73]:
models = {
    'Logistic Regression' : LogisticRegression(max_iter=1000, random_state=42),
    'SVM' : SVC(random_state=42, kernel='linear'),
    'Random Forest' : RandomForestClassifier(random_state=42,n_estimators=100)
}

results = {}
for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = {
        'model' : model,
        'accuracy' : accuracy,
        'predictions' : y_pred,
    }
    print(f"{name} 정확도: {accuracy:.4f}")

Logistic Regression 정확도: 1.0000
SVM 정확도: 1.0000
SVM 정확도: 1.0000
Random Forest 정확도: 1.0000
Random Forest 정확도: 1.0000


In [74]:
best_model_name = max(results.keys(), key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['model']
best_predictions = results[best_model_name]['predictions']

print(f"🥇 최고 성능 모델: {best_model_name}")
print(f"정확도: {results[best_model_name]['accuracy']:.4f}")
print("="*50)

# 분류 리포트
print(f"\n{best_model_name} 상세 성능:")
emotions = ['우울', '불안', '정상']
print(classification_report(y_test, best_predictions, target_names=emotions))

🥇 최고 성능 모델: Logistic Regression
정확도: 1.0000

Logistic Regression 상세 성능:
              precision    recall  f1-score   support

          우울       1.00      1.00      1.00       200
          불안       1.00      1.00      1.00       200
          정상       1.00      1.00      1.00       200

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [75]:
import streamlit as st
@st.cache_resource
def load_models():
    try:
        # 상대 경로를 절대 경로로 변경
        model = joblib.load('../models/best_model.pkl')  
        tfidf = joblib.load('../models/tfidf_vectorizer.pkl') 
        return model, tfidf
    except Exception as e:
        st.error(f"모델 로드 실패: {e}")
        return None, None

In [76]:
import re
def predict_emotion(text):
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    text_tfidf = tfidf.transform([cleaned])
    prediction = best_model.predict(text_tfidf)[0]
    
    return emotion_map[prediction]

In [77]:
# 테스트
test_sentences = [
    "오늘 정말 우울하고 슬퍼요",
    "시험이 너무 걱정되고 불안해요", 
    "날씨가 좋아서 정말 기분이 좋아요",
    "아무것도 하기 싫고 의욕이 없어요",
    "혹시 실수할까봐 계속 걱정돼요"
]

print("🧪 모델 테스트:")
print("="*50)
for sentence in test_sentences:
    emotion = predict_emotion(sentence)
    print(f"문장: '{sentence}'")
    print(f"예측: {emotion}")
    print()

🧪 모델 테스트:
문장: '오늘 정말 우울하고 슬퍼요'
예측: 정상

문장: '시험이 너무 걱정되고 불안해요'
예측: 불안

문장: '날씨가 좋아서 정말 기분이 좋아요'
예측: 정상

문장: '아무것도 하기 싫고 의욕이 없어요'
예측: 우울

문장: '혹시 실수할까봐 계속 걱정돼요'
예측: 불안



In [78]:
# 저장 확인
import os
if os.path.exists('../models/best_model.pkl'):
    print("🎉 모델 파일 저장 성공!")
else:
    print("❌ 모델 파일 저장 실패!")

🎉 모델 파일 저장 성공!


In [79]:
# 💾 새로운 모델 저장
print("💾 새로운 모델 저장 중...")
print("="*50)

# 모델과 TF-IDF 벡터라이저 저장
joblib.dump(best_model, '../models/best_model.pkl')
joblib.dump(tfidf, '../models/tfidf_vectorizer.pkl')

print(f"✅ 모델 저장 완료!")
print(f"📊 모델: {best_model_name}")
print(f"🎯 정확도: {results[best_model_name]['accuracy']:.4f}")
print(f"📁 저장 위치:")
print(f"  - ../models/best_model.pkl")
print(f"  - ../models/tfidf_vectorizer.pkl")
print("\n🎉 이제 웹앱에서 새로운 모델을 사용할 수 있습니다!")

💾 새로운 모델 저장 중...
✅ 모델 저장 완료!
📊 모델: Logistic Regression
🎯 정확도: 1.0000
📁 저장 위치:
  - ../models/best_model.pkl
  - ../models/tfidf_vectorizer.pkl

🎉 이제 웹앱에서 새로운 모델을 사용할 수 있습니다!
✅ 모델 저장 완료!
📊 모델: Logistic Regression
🎯 정확도: 1.0000
📁 저장 위치:
  - ../models/best_model.pkl
  - ../models/tfidf_vectorizer.pkl

🎉 이제 웹앱에서 새로운 모델을 사용할 수 있습니다!


In [80]:
# 🔍 모델 성능 진단
print("🔍 모델 성능 재확인:")
print(f"최고 모델: {best_model_name}")
print(f"정확도: {results[best_model_name]['accuracy']:.4f}")

# 예측 결과 분포 확인
print("\n예측 결과 분포:")
pred_counts = pd.Series(best_predictions).value_counts().sort_index()
print(pred_counts)

print("\n실제 라벨 분포:")
true_counts = pd.Series(y_test).value_counts().sort_index()
print(true_counts)

# 예측 비율 확인
print("\n예측 비율:")
for i in range(3):
    emotion = ['우울', '불안', '정상'][i]
    if i in pred_counts.index:
        ratio = pred_counts[i] / len(y_test) * 100
        print(f"{emotion}: {pred_counts[i]}개 ({ratio:.1f}%)")
    else:
        print(f"{emotion}: 0개 (0.0%)")

🔍 모델 성능 재확인:
최고 모델: Logistic Regression
정확도: 1.0000

예측 결과 분포:
0    200
1    200
2    200
dtype: int64

실제 라벨 분포:
0    200
1    200
2    200
Name: label, dtype: int64

예측 비율:
우울: 200개 (33.3%)
불안: 200개 (33.3%)
정상: 200개 (33.3%)


In [81]:
# 🔍 웹앱과 동일한 전처리로 실제 테스트
print("🧪 웹앱 전처리 함수 테스트")
print("="*50)

def clean_text_webapp(text):
    if not text:
        return ""
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

STOPWORDS = ['이', '그', '저', '것', '의', '가', '을', '를', '에', '에서', '로', '으로', '와', '과', '은', '는', '이다', '하다']

def remove_stopwords_webapp(text):
    words = text.split()
    return ' '.join([word for word in words if word not in STOPWORDS and len(word) > 1])

def predict_webapp_style(text):
    # 웹앱과 동일한 전처리
    cleaned = clean_text_webapp(text)
    processed = remove_stopwords_webapp(cleaned)
    
    print(f"원본: '{text}'")
    print(f"정리: '{cleaned}'")
    print(f"처리: '{processed}'")
    
    if not processed.strip():
        print("⚠️ 전처리 후 빈 문자열!")
        return None, None
    
    # 벡터화 및 예측
    text_tfidf = tfidf.transform([processed])
    prediction = best_model.predict(text_tfidf)[0]
    probability = best_model.predict_proba(text_tfidf)[0]
    
    emotion_map = {0: '우울', 1: '불안', 2: '정상'}
    
    print(f"예측: {emotion_map[prediction]}")
    print(f"확률: {[f'{p:.3f}' for p in probability]}")
    print("-" * 30)
    
    return emotion_map[prediction], probability

# 테스트 문장들
test_sentences = [
    "우울해요",
    "우울하다",
    "오늘 정말 우울해요",
    "우울하고 슬퍼요",
    "불안해요",
    "걱정돼요",
    "기분이 좋아요",
    "행복해요"
]

print("\n📝 테스트 결과:")
for sentence in test_sentences:
    predict_webapp_style(sentence)

🧪 웹앱 전처리 함수 테스트

📝 테스트 결과:
원본: '우울해요'
정리: '우울해요'
처리: '우울해요'
예측: 정상
확률: ['0.322', '0.326', '0.352']
------------------------------
원본: '우울하다'
정리: '우울하다'
처리: '우울하다'
예측: 정상
확률: ['0.322', '0.326', '0.352']
------------------------------
원본: '오늘 정말 우울해요'
정리: '오늘 정말 우울해요'
처리: '오늘 정말 우울해요'
예측: 정상
확률: ['0.091', '0.093', '0.816']
------------------------------
원본: '우울하고 슬퍼요'
정리: '우울하고 슬퍼요'
처리: '우울하고 슬퍼요'
예측: 정상
확률: ['0.322', '0.326', '0.352']
------------------------------
원본: '불안해요'
정리: '불안해요'
처리: '불안해요'
예측: 불안
확률: ['0.123', '0.730', '0.147']
------------------------------
원본: '걱정돼요'
정리: '걱정돼요'
처리: '걱정돼요'
예측: 정상
확률: ['0.322', '0.326', '0.352']
------------------------------
원본: '기분이 좋아요'
정리: '기분이 좋아요'
처리: '기분이 좋아요'
예측: 정상
확률: ['0.044', '0.045', '0.911']
------------------------------
원본: '행복해요'
정리: '행복해요'
처리: '행복해요'
예측: 정상
확률: ['0.322', '0.326', '0.352']
------------------------------


In [82]:
# 🔍 원본 데이터에서 '우울' 관련 단어 분석
print("📊 원본 데이터 '우울' 관련 분석")
print("="*50)

# 원본 데이터 로드
raw_df = pd.read_csv('../data/raw/korean_sentiment_dataset.csv')

# '우울' 단어가 포함된 문장들 찾기
depression_texts = raw_df[raw_df['text'].str.contains('우울', na=False)]
print(f"'우울' 단어 포함 문장 수: {len(depression_texts)}")

print("\n각 라벨별 '우울' 포함 문장:")
for label in [0, 1, 2]:
    emotion = ['우울', '불안', '정상'][label]
    count = len(depression_texts[depression_texts['label'] == label])
    print(f"{emotion}: {count}개")
    
    # 샘플 출력
    samples = depression_texts[depression_texts['label'] == label]['text'].head(3).tolist()
    for i, text in enumerate(samples, 1):
        print(f"  {i}. {text}")
    print()

# '불안' 단어도 비교해보자
anxiety_texts = raw_df[raw_df['text'].str.contains('불안', na=False)]
print(f"\n'불안' 단어 포함 문장 수: {len(anxiety_texts)}")

print("\n각 라벨별 '불안' 포함 문장:")
for label in [0, 1, 2]:
    emotion = ['우울', '불안', '정상'][label]
    count = len(anxiety_texts[anxiety_texts['label'] == label])
    print(f"{emotion}: {count}개")

📊 원본 데이터 '우울' 관련 분석
'우울' 단어 포함 문장 수: 0

각 라벨별 '우울' 포함 문장:
우울: 0개

불안: 0개

정상: 0개


'불안' 단어 포함 문장 수: 319

각 라벨별 '불안' 포함 문장:
우울: 0개
불안: 319개
정상: 0개
'우울' 단어 포함 문장 수: 0

각 라벨별 '우울' 포함 문장:
우울: 0개

불안: 0개

정상: 0개


'불안' 단어 포함 문장 수: 319

각 라벨별 '불안' 포함 문장:
우울: 0개
불안: 319개
정상: 0개


In [83]:
# 🔍 각 감정별 실제 데이터 내용 분석
from collections import Counter
import re

print("📝 각 감정별 실제 문장 샘플")
print("="*60)

raw_df = pd.read_csv('../data/raw/korean_sentiment_dataset.csv')

for label in [0, 1, 2]:
    emotion = ['우울', '불안', '정상'][label]
    emotion_data = raw_df[raw_df['label'] == label]
    
    print(f"\n🎭 {emotion} 라벨 샘플 (10개):")
    samples = emotion_data['text'].head(10).tolist()
    for i, text in enumerate(samples, 1):
        print(f"  {i:2d}. {text}")
    
    print(f"\n{emotion} 라벨에서 자주 나오는 단어들:")
    all_text = ' '.join(emotion_data['text'].astype(str))
    # 간단한 단어 빈도 분석
    words = re.findall(r'[가-힣]+', all_text)
    word_freq = Counter(words)
    top_words = word_freq.most_common(10)
    for word, freq in top_words:
        if len(word) > 1:  # 한 글자 단어 제외
            print(f"    {word}: {freq}회")
    
    print("-" * 60)

📝 각 감정별 실제 문장 샘플

🎭 우울 라벨 샘플 (10개):
   1. 사실은 하루하루가 너무 힘들어요입니다.
   2. 가끔은 마음이 자꾸 가라앉고 어두워요 너무였어요.
   3. 어느 순간부터 아무 이유 없이 눈물이 나요 너무였어요.
   4. 어느 순간부터 기운이 하나도 없고 아무것도 하기 싫어요입니다.
   5. 이상하게도 모든 게 다 나 때문인 것 같아요 괜히했어요.
   6. 이상하게도 일어나기가 너무 힘들어요 항상요.
   7. 요즘 따라 일어나기가 너무 힘들어요 너무했어요.
   8. 그냥 기운이 하나도 없고 아무것도 하기 싫어요 정말했어요.
   9. 요즘 따라 요즘 사는 게 무의미하게 느껴져요 정말하네요.
  10. 솔직히 요즘 사는 게 무의미하게 느껴져요 별 이유 없이.

우울 라벨에서 자주 나오는 단어들:
    너무: 299회
    요즘: 234회
    없고: 197회
    이유: 194회
    힘들어요: 160회
    어느: 120회
    순간부터: 120회
    그냥: 118회
------------------------------------------------------------

🎭 불안 라벨 샘플 (10개):
   1. 그냥 사람들 앞에 서는 게 너무 무서워요 괜히입니다.
   2. 솔직히 계속 가슴이 답답하고 조마조마해요입니다.
   3. 작은 일에도 깜짝깜짝 놀라요 정말했어요.
   4. 솔직히 사람들 앞에 서는 게 너무 무서워요 자꾸.
   5. 어느 순간부터 불안해서 잠을 잘 수가 없어요 항상였어요.
   6. 이상하게도 작은 일에도 깜짝깜짝 놀라요 너무하네요.
   7. 어느 순간부터 사람들 앞에 서는 게 너무 무서워요 매일입니다.
   8. 솔직히 작은 일에도 깜짝깜짝 놀라요 정말요.
   9. 그냥 불안해서 잠을 잘 수가 없어요 자꾸요.
  10. 이상하게도 내일 시험 생각만 해도 가슴이 뛰어요 매일.

불안 라벨에서 자주 나오는 단어들:
    가슴이: 216회
    자꾸: 212회
    요

In [84]:
# 🧪 새로운 모델 실전 테스트 - 문제 문장들 포함
print("🧪 새로운 모델 실전 테스트")
print("="*60)

def test_new_model(text):
    # 전처리 (웹앱과 동일)
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # 불용어 제거
    STOPWORDS = ['이', '그', '저', '것', '의', '가', '을', '를', '에', '에서', '로', '으로', '와', '과', '은', '는', '이다', '하다']
    words = cleaned.split()
    processed = ' '.join([word for word in words if word not in STOPWORDS and len(word) > 1])
    
    if not processed.strip():
        processed = cleaned  # 불용어 제거 후 빈 문자열이면 원본 사용
    
    # 예측
    text_tfidf = tfidf.transform([processed])
    prediction = best_model.predict(text_tfidf)[0]
    probability = best_model.predict_proba(text_tfidf)[0]
    
    emotion_map = {0: '우울', 1: '불안', 2: '정상'}
    
    print(f"입력: '{text}'")
    print(f"처리: '{processed}'")
    print(f"예측: {emotion_map[prediction]} ({probability[prediction]:.3f})")
    print(f"전체 확률: 우울({probability[0]:.3f}) 불안({probability[1]:.3f}) 정상({probability[2]:.3f})")
    print("-" * 50)
    
    return emotion_map[prediction]

# 문제가 있던 문장들과 다양한 테스트 문장들
test_sentences = [
    # 사용자가 문제 제기한 문장
    "밥이 맛있다",
    "날이 화창해요",
    
    # 명확한 감정 표현들
    "오늘 정말 우울해요",
    "너무 불안하고 걱정돼요",
    "기분이 정말 좋아요",
    
    # 간접적 표현들
    "아무것도 하기 싫어요",
    "시험 때문에 스트레스 받아요",
    "친구들과 놀러가서 즐거웠어요",
    
    # 애매한 표현들
    "그냥 그래요",
    "별로 안 좋아요",
    "괜찮은 것 같아요"
]

print("📝 테스트 결과:")
for sentence in test_sentences:
    test_new_model(sentence)

🧪 새로운 모델 실전 테스트
📝 테스트 결과:
입력: '밥이 맛있다'
처리: '밥이 맛있다'
예측: 정상 (0.824)
전체 확률: 우울(0.081) 불안(0.095) 정상(0.824)
--------------------------------------------------
입력: '날이 화창해요'
처리: '날이 화창해요'
예측: 정상 (0.352)
전체 확률: 우울(0.322) 불안(0.326) 정상(0.352)
--------------------------------------------------
입력: '오늘 정말 우울해요'
처리: '오늘 정말 우울해요'
예측: 정상 (0.816)
전체 확률: 우울(0.091) 불안(0.093) 정상(0.816)
--------------------------------------------------
입력: '너무 불안하고 걱정돼요'
처리: '너무 불안하고 걱정돼요'
예측: 우울 (0.833)
전체 확률: 우울(0.833) 불안(0.073) 정상(0.094)
--------------------------------------------------
입력: '기분이 정말 좋아요'
처리: '기분이 정말 좋아요'
예측: 정상 (0.930)
전체 확률: 우울(0.034) 불안(0.035) 정상(0.930)
--------------------------------------------------
입력: '아무것도 하기 싫어요'
처리: '아무것도 하기 싫어요'
예측: 우울 (0.877)
전체 확률: 우울(0.877) 불안(0.060) 정상(0.062)
--------------------------------------------------
입력: '시험 때문에 스트레스 받아요'
처리: '시험 때문에 스트레스 받아요'
예측: 불안 (0.666)
전체 확률: 우울(0.155) 불안(0.666) 정상(0.180)
--------------------------------------------------
입력: '친구들과 놀러가